In [ ]:
import re
import pandas as pd

In [ ]:
def preprocess(data , pattern=24):
    pattern = r"\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s"
    if pattern == 12:
        pattern = r"\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s"
    messages = re.split(pattern, data)[1:]
    dates = re.findall(pattern, data)
    df = pd.DataFrame({
        "user_message": messages,
        "message_date": dates,
    })
    df["date"] = pd.to_datetime(df["message_date"], format="%d/%m/%Y, %H:%M - ")
    users = []
    messages = []
    for message in df["user_message"]:
        # split by : and split to user , message
        entry = re.split("([\w\W]+?):\s",message)
        if entry[1:]: # if it is user message (uname : message)
            users.append(entry[1])
            messages.append(entry[2])
        else: # if it is notification chat without user name
            users.append("notification")
            messages.append(entry[0])
    df["user"] = users
    df["message"] = messages
    df.drop(columns=["user_message"] , inplace=True)

    df["only_date"] = df["date"].dt.date
    df["year"] = df['date'].dt.year
    df["month_num"] = df["date"].dt.month
    df["month"] =  df['date'].dt.month_name()
    df["day"]=  df['date'].dt.day
    df["day_name"] = df["date"].dt.day_name()
    df["hour"] = df["date"].dt.hour
    df["minute"] = df["date"].dt.minute

    period = []
    for hour in df['hour']:
        if hour == 23:
            period.append(str(hour) + "-" + str('00'))
        elif hour == 0:
            period.append(str('00') + "-" + str(hour + 1))
        else:
            period.append(str(hour) + "-" + str(hour + 1))

    df['period'] = period

    return df

In [4]:
insomnia_group_chat = open("WhatsApp Chat with Insomniacs.txt", "r", encoding="utf-8")

In [5]:
charan_chat  = open("WhatsApp Chat with Charan  Rkv.txt", "r", encoding="utf-8")

In [6]:
data = insomnia_group_chat.read()
data

'18/12/2023, 07:54 - Messages and calls are end-to-end encrypted. Only people in this chat can read, listen to, or share them. Learn more.\n02/10/2022, 05:32 - SKY created group "Insomniacs"\n18/12/2023, 07:54 - SKY added you\n18/12/2023, 11:46 - Rafiq Room Mate: null\n18/12/2023, 12:43 - Rafiq Room Mate: Dear Students,\n\nThe EST-Sem-1 of AY 2023-24 is scheduled to commence from 27-12-2023. Those students (Divyangjan/PH) who require scribe facility are informed to register their names at the Exam Section (AB-1, Second floor) with proof on or before 20-12-2023. The scribe facility will be arranged only for the students who register their names at the Exam Section.\n18/12/2023, 12:43 - Rafiq Room Mate: When a candidate taking a test is unable to write or comprehend the question sheet due to dyslexia, visual impairment, or the inability to write due to the loss of fingers or a hand, he is given the option of hiring an amanuensis, also known as a transcriber or scribe, to assist him.\n18/

In [7]:
preprocess(data)

,message_date,date,user,message,only_date,year,month_num,month,day,day_name,hour,minute,period
0,"18/12/2023, 07:54 -",2023-12-18 07:54:00,notification,Messages and calls are end-to-end encrypted. O...,2023-12-18,2023,12,December,18,Monday,7,54,7-8
1,"02/10/2022, 05:32 -",2022-10-02 05:32:00,notification,"SKY created group ""Insomniacs""\n",2022-10-02,2022,10,October,2,Sunday,5,32,5-6
2,"18/12/2023, 07:54 -",2023-12-18 07:54:00,notification,SKY added you\n,2023-12-18,2023,12,December,18,Monday,7,54,7-8
3,"18/12/2023, 11:46 -",2023-12-18 11:46:00,Rafiq Room Mate,null\n,2023-12-18,2023,12,December,18,Monday,11,46,11-12
4,"18/12/2023, 12:43 -",2023-12-18 12:43:00,Rafiq Room Mate,"Dear Students,\n\nThe EST-Sem-1 of AY 2023-24 ...",2023-12-18,2023,12,December,18,Monday,12,43,12-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8384,"12/08/2025, 20:06 -",2025-08-12 20:06:00,Rafiq Room Mate,<Media omitted>\n,2025-08-12,2025,8,August,12,Tuesday,20,6,20-21
8385,"12/08/2025, 20:18 -",2025-08-12 20:18:00,Rafiq Room Mate,<Media omitted>\n,2025-08-12,2025,8,August,12,Tuesday,20,18,20-21
8386,"12/08/2025, 20:18 -",2025-08-12 20:18:00,Rafiq Room Mate,<Media omitted>\n,2025-08-12,2025,8,August,12,Tuesday,20,18,20-21
8387,"12/08/2025, 20:18 -",2025-08-12 20:18:00,Rafiq Room Mate,<Media omitted>\n,2025-08-12,2025,8,August,12,Tuesday,20,18,20-21


In [ ]:
data.split("\n")[0:2]

In [ ]:
# pattern = r"(\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2} (?:AM|PM) - )"
pattern = r"\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s"  # old
# Updated pattern for '03/04/2021, 17:02 - '
pattern = r"\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s"

In [ ]:
messages = re.split(pattern, data)[1:]

In [ ]:
dates = re.findall(pattern, data)

In [ ]:
df = pd.DataFrame({
    "user_message": messages,
    "message_date": dates,
})


In [ ]:
df["message_date"] = pd.to_datetime(df["message_date"], format="%d/%m/%Y, %H:%M - ")

In [ ]:
df.rename(columns={"message_date": "date"}, inplace=True)

In [ ]:
df.head(10)

In [ ]:
users = []
messages = []
count = 0
for message in df["user_message"]:
    entry = re.split("([\w\W]+?):\s",message)
    if entry[1:]: # if it is user message (uname : message)
        users.append(entry[1])
        messages.append(entry[2])
    else: # if it is notification chat without user name
        users.append("group notification")
        messages.append(entry[0])

In [ ]:
df["user"] = users
df["message"] = messages
df.drop(columns=["user_message"] , inplace=True)
df.head()

In [ ]:
df["year"] = df['date'].dt.year

In [ ]:
df["month"] =  df['date'].dt.month_name()

In [ ]:
df["day"]=  df['date'].dt.day

In [ ]:
df["hour"] = df["date"].dt.hour

In [ ]:
df["minute"] = df["date"].dt.minute

In [ ]:
df.head()

In [ ]:
user_name = "Rafiq Room Mate"

In [ ]:
words  = []
for message in df["message"]:
    words.extend(message.split())


In [ ]:
len(words)

In [ ]:
df[df["message"] != "<Media omitted>\n"]